In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import string
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = '본인 데이터 경로'

# Konlpy

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,866 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,074 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install python-mecab-ko

Install mecab-ko
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/example.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.c
mecab-0.996-ko-0.9.2/example/example.c
mecab-0.996-ko-0.9.2/example/thread_test.cpp
mecab-0.996-ko-0.9.2/mecab-config.in
mecab-0.996-ko-0.9.2/man/
mecab-0.996-ko-0.9.2/man/Makefile.am
mecab-0.996-ko-0.9.2/man/mecab.1
mecab-0.996-ko-0.9.2/man/Makefile.in
mecab-0.996-ko-0.9.2/mecab.iss.in
mecab-0.996-ko-0.9.2/config.guess
mecab-0.996-ko-0.9.2/README
mecab-0.996-ko-0.9.2/COPYING
mecab-0.996-ko-0.9.2/CHANGES.md
mecab-0.996-ko-0.9.2/README.md
mecab-0.996-ko-0.9.2/INSTALL
mecab-0.996-ko-0.9.2/config.sub
mecab-0.996-ko-0.9.2/configure.in
mecab-0.996-ko-0.9.2/swig/
mecab-0.996-ko-0.9.2/swig/Makefile
mecab-0.996-ko-0.9.2/swig/version.h.in
mecab-0.996-ko-0.9.2/swig/version.h
mecab-0.996-ko-0.9.2/swig/MeCab.i
mecab-0.996-ko-0.9.2/aclocal.m4
mecab-0.996-ko-0.9.2/LGPL
mecab-0.996-ko-0.9.2/Makefil

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0   913k      0  0:00:01  0:00:01 --:--:-- 6746k
In file included from viterbi.cpp:14:
param.h:30:13: warning: 'Target {anonymous}::lexical_cast(Source) [with Target = std::__cxx11::basic_string<char>; Source = std::__cxx11::basic_string<char>]' defined but not used [-Wunused-function]
   30 | std::string lexical_cast<std::string, std::string>(std::string arg) {
      |             ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
char_property.cpp: In static member function 'static bool MeCab::CharProperty::compile(const char*, const char*, const char*)':
char_property.cpp:194:35: warning: 'void* memset(void*, int, size_t)' clearing an object of non-trivial type 'struct MeCab::CharInfo'; use assignment or value-initialization instead

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
# 형태소 분석기 4개 import
from konlpy.tag import Okt
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from PIL import Image
import numpy as np
import pandas as pd

import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from mecab import MeCab

In [ ]:
hannanum = Hannanum()

# 데이터 전처리

In [ ]:
df['sns'].unique()

array(['블로그', '인스타그램', '트위터', '커뮤니티', '유튜브'], dtype=object)

In [ ]:
pd.set_option('max_colwidth', 1000)
df_youtube = df[df['sns']=='유튜브']
df_blog = df[df['sns']=='블로그']
df_instagram = df[df['sns']=='인스타그램']
df_community = df[df['sns']=='커뮤니티']
df_twitter = df[df['sns']=='트위터']

In [ ]:
df_youtube.head(15) #한 번 보기 ..

In [ ]:
# 정규화
def preprocess(text):
    text = str(text)
    text = text.strip() # 텍스트 시작과 끝에 있는 공백 제거
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\[[^)]*\]', '', text)
    text = re.sub(r'\<[^)]*\>', '', text)
    text = re.sub(r'\{[^)]*\}', '', text)
    text = re.sub(r'\S+\s?뉴스',' ',text) # 뉴스 이름 제거
    text = re.sub(r'\S+\s?기자',' ',text) # 기자 이름 제거
    text = re.sub('\s+', ' ', text)  # 여러 개의 연속 공백을 하나의 공백으로 대체
    text = re.sub(r'\S+@\S+',' ',text)  # 인스타그램 아이디/이메일주소 제거
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\S*\.com\S*', '', text)
    text = re.sub(r'[^\w\s]', ' ', str(text).strip()) # 단어, 공백 아닌 것들 띄어쓰기로 대체
    #text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text) # 문장부호 띄어쓰기로 대체
    text = re.compile('<.*?>').sub('', text) # html태그 제거
    text = re.sub(r'\d',' ',text) # 숫자 제거
    return text

# 불용어 제거
stopwords = ['Copyright','페이스북','유튜브','블로그','인스타그램','홈페이지','instagram','blog','facebook','official','|','팟캐스트','트위터','카카오스토리','Tag','tag','생년월일','바로가기','콜센터']

def delete(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords]  # 불용어가 아닌 단어들로 필터링
    processed_text = ' '.join(filtered_words)  # 단어들을 다시 문자열로 결합하여 전처리된 텍스트 생성
    return processed_text

# pos-tagging
# 한 글자 제외
# 조사, 어미, 접미사, 문장부호 제외
def extract_hannanum(text):
    text_split = hannanum.pos(text,ntags=22)
    text_list=[]
    for word in text_split:
      if len(word[0])>1 and word[1] not in ['JC','JX','JP','EP','EF','EC','ET','XP','XS','S']: #Adverb, Adjective
        text_list.append(word[0])
    return text_list

def finalpreprocess_hannanum(text):
    return extract_hannanum(delete(preprocess(text)))

## youtube

In [ ]:
youtube = df_youtube.copy()

In [ ]:
tqdm.pandas(desc="Preprocessing")
youtube['내용 전처리'] = youtube['내용'].apply(preprocess).progress_apply(delete)

In [ ]:
tqdm.pandas(desc="Preprocessing")
youtube['내용 2차 전처리'] = youtube['내용'].progress_apply(finalpreprocess_hannanum)

In [ ]:
youtube

## instagram

In [ ]:
instagram = df_instagram.copy()

In [ ]:
tqdm.pandas(desc="Preprocessing")
instagram['내용 전처리'] = instagram['내용'].apply(preprocess).progress_apply(delete)

In [ ]:
tqdm.pandas(desc="Preprocessing")
instagram['내용 2차 전처리'] = instagram['내용'].progress_apply(finalpreprocess_hannanum)

In [ ]:
instagram

## blog

In [ ]:
blog = df_blog.copy()

In [ ]:
tqdm.pandas(desc="Preprocessing")
blog['내용 전처리'] = blog['내용'].apply(preprocess).progress_apply(delete)

In [ ]:
tqdm.pandas(desc="Preprocessing")
blog['내용 2차 전처리'] = blog['내용'].progress_apply(finalpreprocess_hannanum)

In [ ]:
blog

## twitter

In [ ]:
twitter = df_twitter.copy()

In [ ]:
tqdm.pandas(desc="Preprocessing")
twitter['내용 전처리'] = twitter['내용'].apply(preprocess).progress_apply(delete)

In [ ]:
tqdm.pandas(desc="Preprocessing")
twitter['내용 2차 전처리'] = twitter['내용'].progress_apply(finalpreprocess_hannanum)

In [ ]:
twitter

## community

In [ ]:
community = df_community.copy()

In [ ]:
tqdm.pandas(desc="Preprocessing")
community['내용 전처리'] = community['내용'].apply(preprocess).progress_apply(delete)

In [ ]:
tqdm.pandas(desc="Preprocessing")
community['내용 2차 전처리'] = community['내용'].progress_apply(finalpreprocess_hannanum)

In [ ]:
community

# concat

In [ ]:
df_sns = pd.concat([youtube,instagram,twitter,blog,community])

In [ ]:
df_sns

In [ ]:
df_sns.to_csv('저장경로/파일명.csv',index=False)